In [1]:
import nemo.collections.asr as nemo_asr
import os
import numpy as np
import torch

[NeMo W 2022-05-05 09:18:56 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


In [2]:
if torch.cuda.is_available() and hasattr(torch.cuda, 'amp') and hasattr(torch.cuda.amp, 'autocast'):
    autocast = torch.cuda.amp.autocast
else:
    @contextlib.contextmanager
    def autocast():
        yield

In [3]:
TOKEN_OFFSET = 100

In [4]:
def load_model(model_path):
    asr_model = nemo_asr.models.EncDecCTCModelBPE.restore_from(model_path)
    asr_model.eval()
    return asr_model

In [5]:
def load_language_model(lm_path, asr_model, alpha=1.0, beta=1.0, beam_width=128):
    local_vocab = asr_model.decoder.vocabulary
    local_vocab = [chr(idx + TOKEN_OFFSET) for idx in range(len(local_vocab))]
    
    beam_search_lm = nemo_asr.modules.BeamSearchDecoderWithLM(
        vocab=local_vocab,
        beam_width=beam_width,
        alpha=alpha,
        beta=beta,
        lm_path=lm_path,
        num_cpus=max(os.cpu_count(), 1),
        input_tensor=False,
    )
    
    return beam_search_lm


def softmax(x):
    e = np.exp(x - np.max(x))
    return e / e.sum(axis=-1).reshape([x.shape[0], 1])




In [6]:
def transcribe(wav_file, logprobs=False):
    if type(wav_file) != list:
        wav_file = [wav_file]
    
    with autocast():
        with torch.no_grad():
                return asr_model.transcribe(wav_file, logprobs=logprobs)
            
#     return asr_model.transcribe(wav_file, logprobs=logprobs)

In [7]:
def transcribe_with_lm(wav_file):
    logits = transcribe(wav_file, True)
    
    probs = [softmax(logits) for logits in logits]
    
    ids_to_text_func = asr_model.tokenizer.ids_to_text
    
    preds = []
    for prob in probs:
        beams_batch = beam_search_lm.forward(log_probs=prob.reshape(1, prob.shape[0], prob.shape[1]), 
                                         log_probs_length=None)
        for beams_idx, beams in enumerate(beams_batch):
            for candidate_idx, candidate in enumerate(beams):
                if ids_to_text_func is not None:
                    # For BPE encodings, need to shift by TOKEN_OFFSET to retrieve the original sub-word ids
                    pred_text = ids_to_text_func([ord(c) - TOKEN_OFFSET for c in candidate[1]])
                else:
                    pred_text = candidate[1]
        preds.append(pred_text)
                
    return preds

In [11]:
asr_model = load_model('/home/harveen/nemo_training/scripts/nemo_experiments/Conformer-CTC-BPE/2022-04-27_10-56-05/checkpoints/Conformer-CTC-BPE.nemo',
                      )

beam_search_lm = load_language_model(lm_path='/home/harveen/ngram/text/ai4b.bin',
                   asr_model = asr_model
                   )

[NeMo I 2022-05-05 09:27:42 mixins:165] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-05-05 09:27:42 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /home/harveen/nemo_training/data/train_combined.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.2
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2022-05-05 09:27:42 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /home/harveen/nemo_training/data/valid_manif

[NeMo I 2022-05-05 09:27:42 features:259] PADDING: 0
[NeMo I 2022-05-05 09:27:42 features:276] STFT using torch
[NeMo I 2022-05-05 09:27:43 save_restore_connector:209] Model EncDecCTCModelBPE was successfully restored from /home/harveen/nemo_training/scripts/nemo_experiments/Conformer-CTC-BPE/2022-04-27_10-56-05/checkpoints/Conformer-CTC-BPE.nemo.


In [9]:

transcribe('/home/harveen/evaluations/taarini_without_numbers/244-F-29_033.wav')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

63.2 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
transcribe_with_lm(['/home/harveen/evaluations/taarini_without_numbers/244-F-29_033.wav'])

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

296 ms ± 2.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
